In [8]:
# Test on images with mass/cal info for each lesion
# Fill table below

# Actual/Pred | mal | ben | not predicted (normal)
# malmass     |     |     |
# malcal      |     |     |
# benmass     |     |     |
# bencal      |     |     |

In [4]:
import numpy as np
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import pickle
from sklearn.metrics import accuracy_score
from scipy.stats import sem
from sklearn.metrics import roc_curve, auc
import cv2
import matplotlib.patches as patches
%matplotlib inline
import pickle

In [17]:
actualfile = "/home/anvit/Desktop/RetinaNet/pytorch-retinanet/csv_annotation_AIIMS_masscal.csv"
#outputfile = "/home/anvit/Desktop/RetinaNet/pytorch-retinanet/output_files/output_retinanet_0.42.txt"
outputfile = "/home/anvit/Desktop/Scripts/FasterRCNN_output_files/output_frcnn_0.46.txt"
predictions = open(outputfile, 'r')
actual = open(actualfile, 'r')
lines_pred = predictions.readlines()
lines_actual = actual.readlines()

actual.close()
predictions.close()

In [18]:
pred_dict = {}
conf_dict = {}
predlabel_dict = {}
for line in lines_pred:
    line = line.rstrip()
    det = line.split(',')
    if(det[0][:-4] not in pred_dict):
        pred_dict[det[0][:-4]] = [[float(det[1]), float(det[2]), float(det[3]), float(det[4])]]
        conf_dict[det[0][:-4]] = [float(det[5])]
        predlabel_dict[det[0][:-4]] = [float(det[6])]
    else:
        pred_dict[det[0][:-4]].append([float(det[1]), float(det[2]), float(det[3]), float(det[4])])
        conf_dict[det[0][:-4]].append(float(det[5]))
        predlabel_dict[det[0][:-4]].append(float(det[6]))

In [19]:
masscal_dict = {}
for xml in os.listdir("/home/anvit/Desktop/Data/AIIMS_Data/Both_labelled/AllSetsXMLs_withmasscal_resized/"):
    tree = ET.parse("/home/anvit/Desktop/Data/AIIMS_Data/Both_labelled/AllSetsXMLs_withmasscal_resized/" + xml)
    root = tree.getroot()
    numob = 0
    
    for obj in root.findall("object"):
        numob += 1
    
    if numob > 0:
        masscal_dict[xml[:-4]] = {}
    elif numob == 0:
        print(xml)
        print("this image has an xml file with no objects")
        continue

    for obj in root.findall("object"):
        if obj[0].text != "MASS" and obj[0].text != "CALCIFICATION":
            continue

        label = obj[0].text
        bbox = obj[4]
        xmin = float(bbox[0].text)
        ymin = float(bbox[1].text)
        xmax = float(bbox[2].text)
        ymax = float(bbox[3].text)
        s = [xmin, ymin, xmax, ymax]
        listTostr = ' '.join([str(elem) for elem in s])
        masscal_dict[xml[:-4]][listTostr] = label

FILE0690471.xml
this image has an xml file with no objects
FILE0597202202202202202202.xml
this image has an xml file with no objects
FILE0819882.xml
this image has an xml file with no objects
FILE0388388.xml
this image has an xml file with no objects
FILE0627453.xml
this image has an xml file with no objects
FILE0841887.xml
this image has an xml file with no objects
FILE03209538.xml
this image has an xml file with no objects
FILE0666845.xml
this image has an xml file with no objects
FILE0567196196196196196196.xml
this image has an xml file with no objects
FILE0787492.xml
this image has an xml file with no objects
FILE1230998.xml
this image has an xml file with no objects
FILE0442780.xml
this image has an xml file with no objects
FILE0649222222222222222222.xml
this image has an xml file with no objects
FILE0484151151151151151151.xml
this image has an xml file with no objects
FILE0628837.xml
this image has an xml file with no objects
FILE003228747.xml
this image has an xml file with no o

In [20]:
#GT
actual_dict = {}
actuallabel_dict = {}
for line in lines_actual:
    line = line.rstrip()
    det = line.split(',')
    path = det[0].split("/")

    if(path[-1][:-4] not in actual_dict):
        try:
            actual_dict[path[-1][:-4]] = [[float(det[1]), float(det[2]), float(det[3]), float(det[4])]]
            actuallabel_dict[path[-1][:-4]] = [float(det[5])]

        except:
            # Normal images not in the dict
            continue
    else:
        actual_dict[path[-1][:-4]].append([float(det[1]), float(det[2]), float(det[3]), float(det[4])])
        actuallabel_dict[path[-1][:-4]].append(float(det[5]))

In [21]:
print(len(actual_dict), len(masscal_dict))
for img in actual_dict:
    if img not in masscal_dict:
        print(img)
for img in masscal_dict:
    if img not in actual_dict:
        print(img)

205 205


In [22]:
print(len(actual_dict), len(actuallabel_dict), len(masscal_dict))
assert len(actual_dict) == len(actuallabel_dict) == len(masscal_dict)

205 205 205


In [23]:
num_mal = 0
num_ben = 0
num_mass = 0
num_cal = 0
num_lesions = 0
# Want: num_mal + num_ben = num_mass + num_cal = num_lesions

num_malmass = 0
num_malcal = 0
num_benmass = 0
num_bencal = 0
# Want: num_malmass + num_malcal = num_mal
# Want: num_benmass + num_bencal = num_ben
# Want: num_malmass + num_benmass = num_mass
# Want: num_malcal + num_bencal = num_cal

for k in actual_dict:
    les = actual_dict[k]
    l = actuallabel_dict[k]
    mc = masscal_dict[k]
    num_lesions += len(les)
    for e in range(len(les)):
        listTostr = ' '.join([str(elem) for elem in les[e]])
        
        if mc[listTostr] == "MASS" and l[e] == 1:
            num_malmass +=1
        elif mc[listTostr] == "CALCIFICATION" and l[e] == 1:
            num_malcal +=1
        elif mc[listTostr] == "MASS" and l[e] == 0:
            num_benmass +=1
        elif mc[listTostr] == "CALCIFICATION" and l[e] == 0:
            num_bencal +=1
        
        
        if l[e] == 1:
            num_mal += 1
        elif l[e] == 0:
            num_ben += 1

    for box in mc:
        if mc[box] == "MASS":
            num_mass += 1
        elif mc[box] == "CALCIFICATION":
            num_cal += 1


assert num_mal + num_ben == num_mass + num_cal == num_lesions
assert num_malcal + num_malmass == num_mal
assert num_bencal + num_benmass == num_ben
assert num_malmass + num_benmass == num_mass
assert num_malcal + num_bencal == num_cal

In [24]:
mal_as_mal = 0
ben_as_ben = 0
ben_as_mal = 0
mal_as_ben = 0

malmass_as_mal = 0
benmass_as_ben = 0
malcal_as_mal = 0
bencal_as_ben = 0

malmass_as_ben = 0
benmass_as_mal = 0
malcal_as_ben = 0
bencal_as_mal = 0

# malmass_unpred = 0
# benmass_unpred = 0
# malcal_unpred = 0
# bencal_unpred = 0

unpredictedmal = 0
unpredictedben = 0
noboxben = 0
noboxmal = 0

nomasscalinfoben = 0
nomasscalinfomal = 0

for img in actual_dict:
    lesions = actual_dict[img]
    labels = actuallabel_dict[img]
    masscalinfo = masscal_dict[img]
    
    try:
        pred_lesions = pred_dict[img]
        pred_labels = predlabel_dict[img]
        pred_confs = conf_dict[img]
    
    except:
        unpredictedmal += sum(labels)
        unpredictedben += len(labels) - sum(labels)
        continue

    for i in range(len(lesions)):
        flag = False

        lesion = lesions[i]
        benormal = labels[i]
        xmin, ymin, xmax, ymax = lesion
        listTostr = ' '.join([str(elem) for elem in lesion])
        massorcal = masscalinfo[listTostr]        
        
        cnt = 0
        for j in range(len(pred_lesions)):
            xminp, yminp, xmaxp, ymaxp = pred_lesions[j]
            predlabel = pred_labels[j]
            midx, midy = (xminp + xmaxp)/2, (yminp + ymaxp)/2
            if midx > xmin and midx < xmax and midy > ymin and midy < ymax:
                if predlabel == 0 and benormal == 1:
                    mal_as_ben += 1
                    if massorcal == "MASS":
                        malmass_as_ben += 1
                    elif massorcal == "CALCIFICATION":
                        malcal_as_ben += 1
                    flag = True
                elif predlabel == 1 and benormal == 0:
                    ben_as_mal += 1
                    if massorcal == "MASS":
                        benmass_as_mal += 1
                    elif massorcal == "CALCIFICATION":
                        bencal_as_mal += 1                    
                    flag = True
                elif predlabel == benormal == 1:
                    mal_as_mal += 1
                    
                    if massorcal == "MASS":
                        malmass_as_mal += 1
                    elif massorcal == "CALCIFICATION":
                        malcal_as_mal += 1
                    flag = True
                elif predlabel == benormal == 0:
                    ben_as_ben += 1
                    
                    if massorcal == "MASS":
                        benmass_as_ben += 1
                    elif massorcal == "CALCIFICATION":
                        bencal_as_ben += 1
                    flag = True
                else:
                    print(predlabel, benormal)
            else:
                cnt += 1
            if flag == True:
                break
        
        if cnt == len(pred_lesions):
            if benormal == 0:
                noboxben += 1
                
            elif benormal == 1:
                noboxmal += 1
                

In [25]:
print(num_ben, num_mal, num_mass, num_cal, num_lesions)

150 138 203 85 288


In [26]:
print(mal_as_mal, mal_as_ben, mal_as_mal + mal_as_ben)

print(num_mal, noboxmal, unpredictedmal, nomasscalinfomal, num_mal - noboxmal - unpredictedmal - nomasscalinfomal)
assert mal_as_mal + mal_as_ben == num_mal - noboxmal - unpredictedmal - nomasscalinfomal

82 0 82
138 15 41.0 0 82.0


In [27]:
print(ben_as_ben, ben_as_mal, ben_as_ben + ben_as_mal)
print(num_ben, noboxben, unpredictedben, nomasscalinfoben, num_ben - noboxben - unpredictedben - nomasscalinfoben)
assert ben_as_ben + ben_as_mal == num_ben - noboxben - unpredictedben - nomasscalinfoben

0 11 11
150 31 108.0 0 11.0


In [28]:
print(malmass_as_mal, num_malmass)
print(malmass_as_ben, num_malmass)
print(noboxmal)
# print(unpred_malmass, num_masses)

print()

print(benmass_as_ben, num_benmass)
print(benmass_as_mal, num_benmass)
print(noboxben)
#print(unpred_benmass, num_benmass)

print()

print(malcal_as_mal, num_malcal)
print(malcal_as_ben, num_malcal)
#print(unpred_malcal, num_malcal)

print()

print(bencal_as_ben, num_bencal)
print(bencal_as_mal, num_bencal)
#print(unpred_bencal, num_bencal)

60 104
0 104
15

0 99
7 99
31

22 34
0 34

0 51
4 51
